In [1]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.5480, 0.1399, 0.3928],
        [0.2062, 0.1328, 0.8493],
        [0.7059, 0.0646, 0.5163],
        [0.9365, 0.1434, 0.7803],
        [0.1458, 0.4249, 0.5384]])


In [4]:
flag = torch.cuda.is_available()
if flag:
    print("CUDA available")
else:
    print("CUDA unavailable")

ngpu= 1
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
print("Driver：", device)
print("GPU version： ",torch.cuda.get_device_name(0))

CUDA available
Driver： cuda:0
GPU version：  GeForce RTX 2080 Ti
